# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000]})



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp): ## q10a
    x = []
    amt = P["Amount Invested"].tolist()
    total = sum(amt)
    for i in amt:
        x_i = i / total
        x.append(x_i)
    beta = P["Beta"].tolist()
    expected = []
    for i in range(len(beta)):
        exp = (rf + (beta[i] * mkt_rp)) * x[i]
        expected.append(exp)
    return sum(expected)

#expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [4]:
def volatility(P, cor): #q10a
    x = []
    amt = P["Amount Invested"].tolist()
    total = sum(amt)
    for i in amt:
        x_i = i / total
        x.append(x_i)
    vol = P["Volatility"].tolist()
    s = 0
    var = 0
    for i in range(len(P)):
        for j in range(i+1, len(P)):
            s += 2 * x[i] * vol[i] * x[i+1] * vol[i+1] * cor
    for i in range(len(vol)):
        #s = s * vol[i] * x[i]
        var = var + (x[i] ** 2) * (vol[i] ** 2)
    sd = (var + s) ** 0.5
    return sd
#volatility(portfolio, -0.2) 

0.24083189157584595

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [5]:
def expected_return2(rf, mkt_rp, mkt_vol, vol): #q10b
    x = vol / mkt_vol
    exp = rf + (x * mkt_rp)
    return exp
#expected_return2(0.02, 0.05, 0.16, 0.2408)

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [6]:
def CAPM_beta(return_path, factors_path):
    df = pd.read_csv(return_path)
    df2 = pd.read_csv(factors_path)
    months = df["month"].tolist()
    years = df["year"].tolist()
    dates = []
    for i in range(len(years)):
        if months[i] < 10:
            dates.append(int(str(years[i]) + "0" + str(months[i])))
        else:
            dates.append(int(str(years[i]) + str(months[i])))
    
    df["date"] = dates
    merged = pd.merge(df, df2, on = "date", how = "left") 
    
    grouped = merged.groupby("CUSIP")

    cusips = []
    exp = []
    mkt = []
    betas = []
    
    for cusip, group in grouped:
        cusips.append(cusip)
        returns = group["RET"].tolist()
        market = group["Mkt-RF"].tolist()
        riskfree = group["RF"].tolist()
        
        exp_ret = sum(returns) / len(returns) 
        exp.append(exp_ret)
    
        mkt_avg = (sum(market) / len(market)) / 100
        mkt.append(mkt_avg)
        
        rf = list(map(lambda x: exp_ret - (x / 100), riskfree))
        
        mkt_return = list(map(lambda x: mkt_avg - (x / 100), riskfree))
        
        rf_mean = sum(rf) / len(rf)
        #exp.extend([rf_mean])
        mkt_mean = sum(mkt_return) / len(mkt_return)
        
        cov = 0
        var = 0
        for i in range(len(market)):
            cov += ((rf[i] - rf_mean) * (market[i] / 100 - mkt_avg))
            var += ((market[i] / 100 - mkt_avg) ** 2)
        covariance = cov / (len(rf) - 1)
        variance = var / (len(market) - 1)
        betas.append(covariance / variance)
    
    output = pd.DataFrame(data = {
        "CUSIP" : cusips,
        "EXP_RETURN" : exp,
        "BETA" : betas,
    })
    
    
    return output
                
#CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV")

CUSIP  EXP_RETURN      BETA
0     00081T10    0.016252  0.002858
1     00108M10    0.011101  0.004684
2     00130H10    0.016152  0.001601
3     00163U10    0.015538  0.002305
4     00182C10    0.014772  0.002804
...        ...         ...       ...
3220   9959910    0.009933  0.005622
3221   9972410    0.012866  0.001858
3222   9972510    0.017177  0.006884
3223   9984910    0.009006 -0.001650
3224   9985510    0.015699  0.006091

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [8]:
def slope(data):
    market_returns = data['BETA'].tolist()
    asset_returns = data['EXP_RETURN'].tolist()
    
    # Fit a linear regression line to the data using np.polyfit
    coefficients = np.polyfit(market_returns, asset_returns, deg=1)
    
    # The slope of the security market line is the coefficient of the market returns
    slope = coefficients[0]
    

    # This should return 0.007981588120535642... with the provided data.
    return slope
#slope(CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV"))

-0.08075006325632034